# Notebook to Reproduce Originality Detection Findings

## Start Testground

```
$ testground daemon
```

## Run Testplans in Docker:
``` 
$ testground run single --plan=coopbc --testcase=bc --runner=local:docker --builder=docker:go --instances=10 --collect
```

### Run Testplans in K8S:
```
$ testground run single --plan=coopbc --testcase=bc --runner=cluster:k8s --builder=docker:go --instances=10 --collect --build-cfg go_proxy_mode=direct
```

# Parsing Outputs

### Find Output Dir

In [85]:
outputs_folder = "c6usir58gb188nj6a000"

In [86]:
import os
import pathlib
from pathlib import Path
import glob

def listdir_r(dirpath):
    paths=[]
    paths.append(dirpath)
    for path in os.listdir(dirpath):
        rpath = os.path.join(dirpath, path)
        if os.path.isdir(rpath):
            subdirs = listdir_r(rpath)
            if not subdirs == []:
                paths.extend(subdirs)
    return paths

all_paths = listdir_r(str(Path(os.getcwd()).parent.absolute()))
all_paths_containing_target = [s for s in all_paths if outputs_folder in s]
target_dir = all_paths_containing_target[0]

print (target_dir)



/home/manjaro/GIT/bc_p2p/src/go/bc_dht/data/outputs/local_docker/coopbc/c6usir58gb188nj6a000


### Find Outputs

In [131]:
file_type = ".out"
search_str = "___UploaderData___"

def check_dir_for_target_lines(search_path):
    target_line = []
    result = []
    targets = 13
    success = False
    for fname in os.listdir(path=search_path):
        if fname.endswith(file_type):
                fo = open(search_path + "/" + fname)
                line = fo.readline()
                line_no = 1
                while line != '' :
                        index = line.find(search_str)
                        if ( index != -1) :
                            print(fname, "[", line_no, ",", index, "] ", line, sep="")
                            success = True
                        line = fo.readline()
                        if success == True and targets > 0:
                            result.append(line)
                            targets -= 1
                        line_no += 1
                fo.close()
    return result

output = []
output_lines = []
for n in all_paths_containing_target:
    output = check_dir_for_target_lines(n)
    if output:
        for l in output:
            print(l)
            output_lines.append(l)


run.out[76,124] {"ts":1639827872994494457,"msg":"","group_id":"single","run_id":"c6usir58gb188nj6a000","event":{"message_event":{"message":"___UploaderData___"}}}

{"ts":1639827872994502773,"msg":"","group_id":"single","run_id":"c6usir58gb188nj6a000","event":{"message_event":{"message":"Instances: 10"}}}

{"ts":1639827872994516729,"msg":"","group_id":"single","run_id":"c6usir58gb188nj6a000","event":{"message_event":{"message":"SimLatency: 100ms"}}}

{"ts":1639827872994526527,"msg":"","group_id":"single","run_id":"c6usir58gb188nj6a000","event":{"message_event":{"message":"SimPeerBandwidth (bytes per second): 1048576"}}}

{"ts":1639827872994532939,"msg":"","group_id":"single","run_id":"c6usir58gb188nj6a000","event":{"message_event":{"message":"SimJitter (ms): 0s"}}}

{"ts":1639827872994542657,"msg":"","group_id":"single","run_id":"c6usir58gb188nj6a000","event":{"message_event":{"message":"SimLoss (%): 0"}}}

{"ts":1639827872994548518,"msg":"","group_id":"single","run_id":"c6usir58gb188nj

### Parse Results to Data Frame

In [211]:
import json
import pandas as pd

#d = {'Instances': [], 'Latency': [], 'Bandwidth': [], 'Jitter': [], 'Loss': [], 'Corrupt': [], 'S2CheckDuration': [], 'OriginalityRatio': [], 'OriginalCombinations': [], 'DhtUploadDuration': [], 'DhtQueryDuration': []}

kvs_dict = {}
kvs_dict["RunID"] = outputs_folder
for o in output_lines:
    m = json.loads(o)["event"]["message_event"]["message"]
    kv = m.split(": ", 1)
    if len(kv) > 1:
        kvs_dict[kv[0]] = kv[1]
print(kvs_dict)

pd.DataFrame.from_dict(kvs_dict, orient='index')


{'RunID': 'c6usir58gb188nj6a000', 'Instances': '10', 'SimLatency': '100ms', 'SimPeerBandwidth (bytes per second)': '1048576', 'SimJitter (ms)': '0s', 'SimLoss (%)': '0', 'SimCorrupt (%)': '0', 'S2CheckDuration': '4.964329012s', 'OriginalityRatio (RO)': '0.7', 'UploadedOriginalCombinations': '14', 'UploadDuration': '5.354883614s', 'DhtQueryDuration': '6.578146ms'}


,0
RunID,c6usir58gb188nj6a000
Instances,10
SimLatency,100ms
SimPeerBandwidth (bytes per second),1048576
SimJitter (ms),0s
SimLoss (%),0
SimCorrupt (%),0
S2CheckDuration,4.964329012s
OriginalityRatio (RO),0.7
UploadedOriginalCombinations,14


### Accumulate all Runs 